# T1614.001 - System Language Discovery
Adversaries may attempt to gather information about the system language of a victim in order to infer the geographical location of that host. This information may be used to shape follow-on behaviors, including whether the adversary infects the target and/or attempts specific actions. This decision may be employed by malware developers and operators to reduce their risk of attracting the attention of specific law enforcement agencies or prosecution/scrutiny from other entities.(Citation: Malware System Language Check)

There are various sources of data an adversary could use to infer system language, such as system defaults and keyboard layouts. Specific checks will vary based on the target and/or adversary, but may involve behaviors such as [Query Registry](https://attack.mitre.org/techniques/T1012) and calls to [Native API](https://attack.mitre.org/techniques/T1106) functions.(Citation: CrowdStrike Ryuk January 2019) 

For example, on a Windows system adversaries may attempt to infer the language of a system by querying the registry key <code>HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\Nls\Language</code> or parsing the outputs of Windows API functions <code>GetUserDefaultUILanguage</code>, <code>GetSystemDefaultUILanguage</code>, <code>GetKeyboardLayoutList</code> and <code>GetUserDefaultLangID</code>.(Citation: Darkside Ransomware Cybereason)(Citation: Securelist JSWorm)(Citation: SecureList SynAck Doppelgänging May 2018)

On a macOS or Linux system, adversaries may query <code>locale</code> to retrieve the value of the <code>$LANG</code> environment variable.

## Atomic Tests

### Atomic Test #1 - Discover System Language by Registry QueryIdentify System language by querying the registry on an endpoint. 

Upon successful execution, result in number format can be looked up to correlate the language.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg query HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\Nls\Language
```

In [ ]:
Invoke-AtomicTest T1614.001 -TestNumbers 1

### Atomic Test #2 - Discover System Language with chcpIdentify System language with the chcp command.

Upon successful execution, result in number format can be looked up to correlate the language.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
chcp
```

In [ ]:
Invoke-AtomicTest T1614.001 -TestNumbers 2

### Atomic Test #3 - Discover System Language with localeIdentify System language with the `locale` command.

Upon successful execution, the output will contain the environment variables that indicate
the 5 character locale that can be looked up to correlate the language and territory.
**Supported Platforms:** linux#### Attack Commands: Run with `sh`
```sh
locale
```

In [ ]:
Invoke-AtomicTest T1614.001 -TestNumbers 3

### Atomic Test #4 - Discover System Language with localectlIdentify System language with the `localectl` command.

Upon successful execution, the key `System Locale` from the output will contain the
`LANG` environment variable that has the 5 character locale result that can be looked
up to correlate the language and territory.
**Supported Platforms:** linux#### Attack Commands: Run with `sh`
```sh
localectl status
```

In [ ]:
Invoke-AtomicTest T1614.001 -TestNumbers 4

### Atomic Test #5 - Discover System Language by locale file
Identify System language with the by reading the locale configuration file.

The locale configuration file contains the `LANG` environment variable which
will contain the 5 character locale that can be looked up to correlate the
language and territory.

**Supported Platforms:** linux
#### Dependencies:  Run with `sh`!
##### Description: Check the location of the locale configuration file.

##### Check Prereq Commands:
```sh
[ -f /etc/locale.conf ] || [ -f /etc/default/locale ] && exit 0 || exit 1

```
##### Get Prereq Commands:
```sh
echo "Test only valid for systems that have locale file"

```

In [ ]:
Invoke-AtomicTest T1614.001 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
[ -f /etc/locale.conf ] && cat /etc/locale.conf || cat /etc/default/locale
```

In [ ]:
Invoke-AtomicTest T1614.001 -TestNumbers 5

### Atomic Test #6 - Discover System Language by Environment Variable Query
Identify System language by checking the environment variables

Upon successful execution, the 5 character locale result can be looked up to
correlate the language and territory. Environment query commands are likely
to run with a pattern match command e.g. `env | grep LANG`

Note: `env` and `printenv` will usually provide the same results. `set` is
also used as a builtin command that does not generate syscall telemetry but
does provide a list of the environment variables.

**Supported Platforms:** linux
#### Dependencies:  Run with `sh`!
##### Description: Check if printenv command exists on the machine

##### Check Prereq Commands:
```sh
[ -x "$(command -v printenv)" ] && exit 0 || exit 1

```
##### Get Prereq Commands:
```sh
echo "printenv command does not exist"
exit 1

```

In [ ]:
Invoke-AtomicTest T1614.001 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
env | grep LANG
printenv LANG
set | grep LANG
```

In [ ]:
Invoke-AtomicTest T1614.001 -TestNumbers 6

## Detection
System and network discovery techniques normally occur throughout an operation as an adversary learns the environment. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities based on the information obtained.

Monitor processes and command-line arguments for actions that could be taken to gather system language information. This may include calls to various API functions and interaction with system configuration settings such as the Windows Registry.